In [1]:
from collections import defaultdict
from glob import glob
import os

import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm

import seaborn as sns

plt.style.use('seaborn-white')
import matplotlib as mpl
from matplotlib import gridspec

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

Load data

In [3]:
DATA_DIR = './tutorial-data'

SEI_DIR = os.path.join(DATA_DIR, 'sei_data')
PREDICTIONS_DIR = os.path.join(DATA_DIR, '1000G_EUR_Phase3')
GENE_DATA = os.path.join(DATA_DIR, 'gene_information')

SUMSTATS_FILE = os.path.join(DATA_DIR, 'BreastCancer_Michailidou_Nature_2017.sumstats.gz')

NB_OUT = os.path.join(DATA_DIR, 'nb_outputs')
HB_OUT = os.path.join(DATA_DIR, 'hb_outputs')

Breast cancer summary statistics file, from [Michailidou et al. (2017)](https://www.nature.com/articles/nature24284) paper. 

In [4]:
sumstats_df = pd.read_csv(SUMSTATS_FILE, sep='\t')
sumstats_df.set_index(['SNP'], inplace=True)
sumstats_df.head()

,A1,A2,Z,N
SNP,,,,
rs201725126,T,G,0.503,228951.0
rs200579949,A,G,0.503,228951.0
rs75454623,A,G,-1.043,228951.0
rs199856693,G,A,0.443,228951.0
rs78601809,T,G,0.407,228951.0


In [5]:
snps = sumstats_df.index.tolist()
len(snps)

8191810

Load the corresponding Sei predictions for these variants

In [7]:
def _add_chr17(mat):
    use_mat = np.zeros(
        (mat.shape[0] + 7, mat.shape[1]))
    use_mat[7:, :] = mat
    return use_mat[:, :]

rowlabels = []
zscores = []
sei_scores = []
for fp in sorted(glob(os.path.join(PREDICTIONS_DIR, 'vcfs', '*.vcf'))):
    fn = os.path.basename(fp).rsplit('.', 1)[0]
    print(fn)
    tag = fn.split('.')[0]
    score_fn = '{0}.ref_predictions.h5.diffs.npy'.format(fn)
    
    df = pd.read_csv(fp, sep='\t', header=None)
    ixs = np.arange(len(df))
        
    df['ix'] = ixs
    df.set_index([2], inplace=True)
    subset_df = df.loc[df.index.intersection(snps)]
    subset_ixs = subset_df['ix'].tolist()

    sumstats_z = sumstats_df.loc[subset_df.index.tolist()]
    subset_zscores = sumstats_z['Z'].tolist()

    sc_scores = np.load(os.path.join(
        PREDICTIONS_DIR, 'sequence_class_scores', score_fn))
    
    if tag == '17':
        sc_scores = _add_chr17(sc_scores)
    
    rowlabels.append(subset_df)
    zscores.append(subset_zscores)
    sei_scores.append(sc_scores[subset_ixs])
    
rowlabels = pd.concat(rowlabels)
zscores = np.hstack(zscores)
sei_scores = np.vstack(sei_scores)

1.1000G.EUR.QC.vcf.swap
10.1000G.EUR.QC.vcf.swap
11.1000G.EUR.QC.vcf.swap
12.1000G.EUR.QC.vcf.swap
13.1000G.EUR.QC.vcf.swap
14.1000G.EUR.QC.vcf.swap
15.1000G.EUR.QC.vcf.swap
16.1000G.EUR.QC.vcf.swap
17.1000G.EUR.QC.vcf.swap
18.1000G.EUR.QC.vcf.swap
19.1000G.EUR.QC.vcf.swap
2.1000G.EUR.QC.vcf.swap
20.1000G.EUR.QC.vcf.swap
21.1000G.EUR.QC.vcf.swap
22.1000G.EUR.QC.vcf.swap
3.1000G.EUR.QC.vcf.swap
4.1000G.EUR.QC.vcf.swap
5.1000G.EUR.QC.vcf.swap
6.1000G.EUR.QC.vcf.swap
7.1000G.EUR.QC.vcf.swap
8.1000G.EUR.QC.vcf.swap
9.1000G.EUR.QC.vcf.swap


In [8]:
rowlabels['ix'] = list(range(len(rowlabels)))
rowlabels.columns = ['chrom', 'pos', 'ref', 'alt', 'ix']
rowlabels.head()

,chrom,pos,ref,alt,ix
rs200579949,1,13118,A,G,0
rs199856693,1,14933,G,A,1
rs374029747,1,15774,G,A,2
rs199745162,1,16949,A,C,3
rs141149254,1,54490,G,A,4


Saved to the following paths (for ease of loading the data again at a later point in time)

In [10]:
#np.save(os.path.join(NB_OUT, 'brca_gwas.zscores.npy'), zscores)
#np.save(os.path.join(NB_OUT, 'brca_gwas.sei_scores.npy'), sei_scores)
#rowlabels.to_csv(os.path.join(NB_OUT, 'brca_gwas.rowlabels.tsv'), sep='\t')

In [11]:
#rowlabels = pd.read_csv(os.path.join(NB_OUT, 'brca_gwas.rowlabels.tsv'), sep='\t')
#zscores = np.load(os.path.join(NB_OUT, 'brca_gwas.zscores.npy'))
#sei_scores = np.load(os.path.join(NB_OUT, 'brca_gwas.sei_scores.npy'))

In [12]:
with open(os.path.join(NB_OUT, 'brca_gwas_snps.bed'), 'w+') as fh:
    for row in rowlabels.itertuples():
        chrom = 'chr{0}'.format(row.chrom)
        s = row.pos
        e = row.pos + 1
        vid = '{0}_{1}_{2}_{3}_{4}'.format(row.chrom, row.pos, row.ref, row.alt, row.ix)
        fh.write('{0}\t{1}\t{2}\t{3}\n'.format(
            chrom, s, e, vid))

In [14]:
ix_to_variant = {}
variant_to_ix = {}
for row in rowlabels.itertuples():
    ix_to_variant[row.ix] = (row.chrom, row.pos, row.ref, row.alt, row.Index)
    variant_to_ix[(row.chrom, row.pos, row.ref, row.alt)] = row.ix

This is a good time to look into a tool called [`bedtools`](https://bedtools.readthedocs.io/en/latest/). We will use bedtools to pull different genome annotation information into our analysis.

It can be installed in your conda environment:

```
conda install -c bioconda bedtools
```

We provide a number of annotations for you. For example, we generated `brca_gwas_snps.sc_annot.bed` by running

```
cd ./tutorial-data/nb_outputs

sort -k1,1 -k2,2n brca_gwas_snps.bed > sort.brca_gwas_snps.bed

bedtools intersect -a ./sort.brca_gwas_snps.bed -b ../sei_data/sc_annotation.bed -wa -wb > brca_gwas_snps.sc_annot.bed
```

In [15]:
sc_df = pd.read_csv(os.path.join(NB_OUT, 'brca_gwas_snps.sc_annot.bed'), sep='\t', header=None)
sc_df.head()

,0,1,2,3,4,5,6,7
0,chr1,13118,13119,1_13118_A_G_0,chr1,12151,13650,0
1,chr1,14933,14934,1_14933_G_A_1,chr1,14451,15450,0
2,chr1,15774,15775,1_15774_G_A_2,chr1,15451,15850,27
3,chr1,16949,16950,1_16949_A_C_3,chr1,16851,16950,2
4,chr1,54490,54491,1_54490_G_A_4,chr1,54351,54650,24


map the variant to the sequence class annotation

In [16]:
variant_sc = {}
for row in sc_df.itertuples():
    chrom, pos, ref, alt, _ = row._4.split('_')
    variant_sc[(int(chrom), int(pos), ref, alt)] = row._8

you can then use this information to summarize the matrix of Sei sequence class scores we have into single vectors of data:

In [17]:
scores_vec = []
sc_annotvec = []
variant_labels = []
for ix, row in enumerate(sei_scores):
    (chrom, pos, ref, alt, rsid) = ix_to_variant[ix]
    vid = (chrom, pos, ref, alt)
    variant_labels.append(vid)
    if vid not in variant_sc:
        sc_annotvec.append(-1)
        rix = np.argmax(np.abs(row))
        scores_vec.append(row[rix])
    else:
        sc = variant_sc[vid]
        sc_annotvec.append(sc)
        scores_vec.append(row[sc])

In [18]:
scores_vec = np.array(scores_vec)
sc_annotvec = np.array(sc_annotvec)

This is a good time to experiment with the data at the variant-level! Summarize, filter, slice/dice the information, make some visualizations, etc.

In [ ]:
# TODO MORE CODE HERE

Once the variants have been prioritized based on their predicted functional impact, it is useful to then map those variants to the genes they may affect. If we know what genes are affected by these mutations, we can get a sense of what biological pathways may be dysregulated. We might ask if these pathways seem cancer-relevant or not.

Below we've included some BED files with gene mappings:

- `pcHiC.oes_geneanno.hg19.bed` contains promoter-capture Hi-C mappings from [Javierre et al. (2016)](https://doi.org/10.1016/j.cell.2016.09.037). 
- `geneanno.hg19.bed` contains the closest gene mapping based on FANTOM CAGE transcription start site (TSS) coordinates

`pcHiC.oes_geneanno.hg19.bed` doesn't have full coverage of variant -> gene coordinates, so you can first map variants with promoter-capture Hi-C data to the appropriate gene(s) and then use the closest gene for the rest.

In [19]:
pchic_df = pd.read_csv(os.path.join(
    GENE_DATA, 'pcHiC.oes_geneanno.hg19.bed'), sep='\t', header=None)
pchic_df.head()

,0,1,2,3,4,5
0,chr1,82860,84549,20,.,RP11-396F22.1;CPNE8
1,chr1,90820,92086,23,.,AC069287.1;ODF3;PRDM7;RP11-304M2.5;SCGB1C1
2,chr1,759752,761673,197,.,TBP;PSMB1
3,chr1,772248,774390,199,.,RP11-63E5.6
4,chr1,831395,848668,218,RP11-54O7.16;RP11-54O7.1,AL645608.1;SAMD11;RP11-54O7.3


In [20]:
closest_genes_df = pd.read_csv(os.path.join(
    GENE_DATA, 'geneanno.hg19.bed'), sep='\t', header=None)
closest_genes_df.head()

,0,1,2,3,4,5
0,chr1,29554,29555,RP11-34P13.3,ENSG00000243485,+
1,chr1,36081,36082,FAM138A,ENSG00000237613,-
2,chr1,69091,69092,OR4F5,ENSG00000186092,+
3,chr1,133723,133724,RP11-34P13.7,ENSG00000238009,-
4,chr1,139379,139380,AL627309.1,ENSG00000237683,-


In [ ]:
# TODO MORE CODE HERE

Once you have the closest gene assignments, try running [module detection at HumanBase](https://hb.flatironinstitute.org/module/) to see what pathways are impacted by these genes!

In [ ]:
# TODO MORE CODE HERE AS NEEDED